In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ximenml0613/raw/master/poem_train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ximenml0613/raw/master/poem_test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
train_df

In [ ]:
test_df = pd.read_csv("test.csv", encoding="utf-8")
test_df

In [ ]:
# Series.replace(字典)
# {"李白":0, "杜甫":1}
# Series.unique()/Series.value_counts()
u = train_df["作者"].unique()
trans = {n:i for i, n in enumerate(u)}
trans_r = {i:n for i, n in enumerate(u)}
print(trans)
print(trans_r)
y_train = train_df["作者"].replace(trans)
y_test = test_df["作者"].replace(trans)

{'李白': 0, '杜甫': 1, '白居易': 2}
{0: '李白', 1: '杜甫', 2: '白居易'}


In [ ]:
import jieba
# [TODO]: check標點符號/換行sklearn會不會幫我搞定
c = train_df["內容"][0]
" ".join(jieba.cut(c))

In [ ]:
# apply: 設定一個流程, 對每一格都執行這流程
def poemcut(c):
    return " ".join(jieba.cut(c))
x_train = train_df["內容"].apply(poemcut)
x_test = test_df["內容"].apply(poemcut)
x_test

0      日照 香爐生 紫煙 ， 遙看 瀑布 掛 前川 。 飛流 直下 三千尺 ， 疑是 銀河 落九天 。
1     朝辭 白帝 彩雲間 ， 千裡 江陵 一日 還 。 兩岸 猿聲 啼 不住 ， 輕舟 已過 萬 ...
2      李白 乘舟 將欲行 ， 忽聞 岸上 踏歌 聲 。 桃花潭水 深 千尺 ， 不及 汪倫送 我情 。
3       故人 西辭黃鶴樓 ， 煙花 三月 下揚州 。 孤帆 遠影 碧空 儘 ， 唯見長 江天 際流 。
4                 危樓 高 百尺 ， 手可摘 星辰 。 不敢 高聲語 ， 恐驚 天上 人 。
5                  床前 明月光 ， 疑是 地上 霜 。 舉頭 望明月 ， 低頭思 故鄉 。
6     天門 中斷 楚江 開 ， 碧水 東流 至此 回 。 兩岸 青山 相對 出 ， 孤帆 一片 日...
7               眾鳥 高 飛 儘 ， 孤雲獨 去 閒 。 相看 兩不厭 ， 隻 有 敬亭山 。
8     鳳凰 台上 鳳凰遊 ， 鳳去 台空江 自流 。 吳宮 花草 埋 幽徑 ， 晉代 衣冠 成古丘...
9     渡遠 荊門外 ， 來 從 楚國遊 。 山 隨 平野 儘 ， 江入 大荒 流 。 月 下 飛天...
10    百川 日東流 ， 客去 亦 不息 。 我 生苦 漂 蕩 ， 何時 有 終極 。 讚 公 釋 ...
11    細泉 兼 輕冰 ， 沮洳 棧道 濕 。 不辭 辛苦 行 ， 迫此 短景急 。 石門 雪雲隘 ...
12    首路 栗亭 西 ， 尚想 鳳凰村 。 季冬 攜 童稚 ， 辛苦 赴 蜀門 。 南登 木皮 嶺...
13    落日 在 簾 鉤 ， 溪邊 春事幽 。 芳菲 緣岸 圃 ， 樵 爨 倚灘 舟 。 啅 雀 爭...
14    竇侍 禦 ， 驥 之子 ， 鳳之雛 。 年 未 三十 忠義俱 ， 骨鯁 絕代 無 。 炯 如...
15    百草 競春華 ， 麗春應 最勝 。 少須 好 顏色 ， 多漫枝條 剩 。 紛紛 桃李 枝 ，...
16    何年 顧虎頭 ， 滿壁畫 瀛州 。 赤 日 石林 氣 ， 青天 江海 流 。 錫飛常 近鶴 ...
17    野寺 隱喬木 ， 山僧 高下 居 。 石門 日色異 ， 絳 氣橫 扶疏 。 窈窕 入

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [ ]:
# print(x_train_count)
# check: \r\n, 標點
# 出現KeyError: ok! 代表不在裡面
# vec.vocabulary_["，"]

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

0.8

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(y_test, pre)
pd.DataFrame(mat,
             columns=[n + "(預測值)" for n in u],
             index=[n + "(正確值)" for n in u])

,李白(預測值),杜甫(預測值),白居易(預測值)
李白(正確值),8,1,1
杜甫(正確值),1,8,1
白居易(正確值),1,1,8


In [43]:
p = input("輸入:")
# 必須是整批 [分詞1, 分詞2....]
count = vec.transform([poemcut(p)])
pre = clf.predict(count)[0]
print("是", trans_r[pre], "寫的")
proba = clf.predict_proba(count)[0]
for n, p in zip(u, proba):
    print(n, ":", p)

輸入:空無一人的大街 闖入無人婚紗店 為妳披上雪白誓言 世界已灰飛煙滅 而愛矗立高樓間 你是真的 或是我的 幻覺  時光遺忘的背面 獨坐殘破的台階 哪個亂世沒有離別 天空和我的中間 只剩傾盆的思念 如果相識 不能相戀 是不是還不如擦肩  在失去你的風景裡面 你卻佔據了每一條街 一步步曾經 一步步想念 在腳下蔓延 在充滿你的回憶裡面 我獨自流浪海角天邊 一步步走過 當時心願  格林威治大鐘前 歸零超載的傷悲 揹著我和我的諾言 一起計畫的路線 對照孤單的旅店 一聲晚安 卻又喚醒 淚腺  時代廣場的跨年 頤和花季的藍天 數著願望在你指尖 當時有多少心願 就有多少的殘缺 如果後悔 不能後退 是不是就只能往前  在失去你的風景裡面 你卻佔據了每一條街 一步步曾經 一步步想念 在腳下蔓延 在充滿你的回憶裡面 我獨自流浪海角天邊 一步步走過 當時心願  生如浮萍般卑微 愛卻蒼穹般壯烈 我要為你爬上最險山嶽 走過最崎嶇眷戀 一步一步穿越  在失去你的風景裡面 你卻佔據了整個世界 每一張相片 每一個房間 每一滴眼淚 在充滿你的回憶裡面 我獨自一人和眼淚周旋 一步步走向 孤單的明天  也許在來生的某個明天 我們能再寫 新的情節 一步步完成 當時心願 一步步完成 最美殘缺
是 李白 寫的
李白 : 0.8655964437489705
杜甫 : 0.000565879828750617
白居易 : 0.13383767642228744
